In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import ipywidgets as widgets
from savify import Savify
from savify.types import Type, Format, Quality
from savify.utils import PathHolder
import logging
from tqdm import tqdm
from IPython.display import display
import plotly.graph_objects as go
import plotly.subplots

%matplotlib notebook

# get the client id and client secret from the text file
with open('spotify_credentials.txt') as f:
    client_id = f.readline().strip().split(' ')[1]
    client_secret = f.readline().strip().split(' ')[1]
    uri = f.readline().split(' ')[1][:-1]

uri = 'http://localhost:8000'

# set the environment variables
os.environ['SPOTIPY_CLIENT_ID'], os.environ['SPOTIFY_CLIENT_ID'] = client_id, client_id
os.environ['SPOTIPY_CLIENT_SECRET'], os.environ['SPOTIFY_CLIENT_SECRET'] = client_secret, client_secret
os.environ['SPOTIPY_REDIRECT_URI'], os.environ['SPOTIFY_REDIRECT_URI'] = uri, uri

#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

# just add all the scopes
scopes = ['user-library-read',
            'user-read-recently-played',
            'user-top-read',
            'user-follow-read',
            'user-read-playback-position',
            'user-read-playback-state',
            'user-read-currently-playing',
            'user-modify-playback-state',
            'user-read-private',
            'playlist-read-private',
            'playlist-read-collaborative',
            'playlist-modify-public',
            'playlist-modify-private']

username = '1260351083'

token = spotipy.util.prompt_for_user_token(username, scopes)

if token:
    sp = spotipy.Spotify(auth=token)
    saved_tracks_resp = sp.current_user_saved_tracks(limit=50)
else:
    print('Couldn\'t get token for that username')

path_holder = PathHolder(downloads_path='downloads')
logger = logging.getLogger('savify')
s = Savify(path_holder=path_holder, logger=logger, download_format=Format.MP3)

In [6]:
playlists = sp.user_playlists(username)
playlists['items'][1]['name']

'October 2023'

In [7]:
user = sp.user(username)
user

{'display_name': 'Landry Bulls',
 'external_urls': {'spotify': 'https://open.spotify.com/user/1260351083'},
 'href': 'https://api.spotify.com/v1/users/1260351083',
 'id': '1260351083',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b829ba38eeccc9a44b22c74f931',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee859ba38eeccc9a44b22c74f931',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:1260351083',
 'followers': {'href': None, 'total': 80}}

In [8]:
sp.user = user

In [9]:
sp.current_playback()

{'device': {'id': 'b1a5d7096e2a9c1339b49f029db792e7f6c0aa4b',
  'is_active': True,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'C02D91JSML86',
  'supports_volume': True,
  'type': 'Computer',
  'volume_percent': 100},
 'shuffle_state': True,
 'repeat_state': 'off',
 'timestamp': 1697575388029,
 'context': {'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1F5p3rmiWPIYgZ'},
  'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1F5p3rmiWPIYgZ',
  'type': 'playlist',
  'uri': 'spotify:playlist:37i9dQZF1F5p3rmiWPIYgZ'},
 'progress_ms': 72532,
 'item': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1ky3oGuE5XOsOzqiFEGwqR'},
     'href': 'https://api.spotify.com/v1/artists/1ky3oGuE5XOsOzqiFEGwqR',
     'id': '1ky3oGuE5XOsOzqiFEGwqR',
     'name': 'Mansur Brown',
     'type': 'artist',
     'uri': 'spotify:artist:1ky3oGuE5XOsOzqiFEGwqR'}],
   'available_markets': ['AR',
    'AU',
  

In [10]:
# pause the current playback
sp.pause_playback()

In [11]:
# start the current playback
sp.start_playback()

In [12]:
# ask if the song is playing
sp.current_user_playing_track()['is_playing']

True

In [ ]:
# skip the current playback
sp.next_track()

In [13]:
# get the current playback
sp.current_playback()

{'device': {'id': 'b1a5d7096e2a9c1339b49f029db792e7f6c0aa4b',
  'is_active': True,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'C02D91JSML86',
  'supports_volume': True,
  'type': 'Computer',
  'volume_percent': 100},
 'shuffle_state': True,
 'repeat_state': 'off',
 'timestamp': 1697575461672,
 'context': {'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1F5p3rmiWPIYgZ'},
  'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1F5p3rmiWPIYgZ',
  'type': 'playlist',
  'uri': 'spotify:playlist:37i9dQZF1F5p3rmiWPIYgZ'},
 'progress_ms': 77706,
 'item': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1ky3oGuE5XOsOzqiFEGwqR'},
     'href': 'https://api.spotify.com/v1/artists/1ky3oGuE5XOsOzqiFEGwqR',
     'id': '1ky3oGuE5XOsOzqiFEGwqR',
     'name': 'Mansur Brown',
     'type': 'artist',
     'uri': 'spotify:artist:1ky3oGuE5XOsOzqiFEGwqR'}],
   'available_markets': ['AR',
    'AU',
  

In [14]:
# get the features of the current playback
sp.audio_features(sp.current_playback()['item']['id'])

[{'danceability': 0.665,
  'energy': 0.721,
  'key': 5,
  'loudness': -9.861,
  'mode': 0,
  'speechiness': 0.0489,
  'acousticness': 0.0109,
  'instrumentalness': 0.912,
  'liveness': 0.309,
  'valence': 0.141,
  'tempo': 100.085,
  'type': 'audio_features',
  'id': '2bUYQj4ev1LUHX8mAlowyP',
  'uri': 'spotify:track:2bUYQj4ev1LUHX8mAlowyP',
  'track_href': 'https://api.spotify.com/v1/tracks/2bUYQj4ev1LUHX8mAlowyP',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2bUYQj4ev1LUHX8mAlowyP',
  'duration_ms': 382760,
  'time_signature': 4}]

In [15]:
# get the halloween party playlist
playlists = sp.user_playlists(username)
#playlists['items'][1]['name']
elm = [x for x in playlists['items'] if x['name'] == '89Elm'][0]
elm

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1nNxmkhYsOpa6Ux9MFJLoc'},
 'href': 'https://api.spotify.com/v1/playlists/1nNxmkhYsOpa6Ux9MFJLoc',
 'id': '1nNxmkhYsOpa6Ux9MFJLoc',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b273841c8f567b42b8b9a1aa54ecab67616d0000b273b3994c94dfb241923664bb4dab67616d0000b273b47d309281c66820b7137f5dab67616d0000b273de437d960dda1ac0a3586d97',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b273841c8f567b42b8b9a1aa54ecab67616d0000b273b3994c94dfb241923664bb4dab67616d0000b273b47d309281c66820b7137f5dab67616d0000b273de437d960dda1ac0a3586d97',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b273841c8f567b42b8b9a1aa54ecab67616d0000b273b3994c94dfb241923664bb4dab67616d0000b273b47d309281c66820b7137f5dab67616d0000b273de437d960dda1ac0a3586d97',
   'width': 60}],
 'name': '89Elm',
 'owner': {'display_na

In [16]:
elm['tracks']['total']

191

In [17]:
# get the tracks of the playlist
# cant go any higher than 100
tracks = sp.playlist_tracks(elm['id'])

# get the features of the tracks and put them in a dataframe
features = []
for track in tracks['items']:
    feature_data = sp.audio_features(track['track']['id'])[0]
    feature_data['link'] = track['track']['external_urls']['spotify']
    features.append(feature_data)

elm_features = pd.DataFrame(features)
display(elm_features.head())


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,link
0,0.771,0.687,11,-9.814,1,0.0693,0.0318,0.000225,0.890,0.783,118.392,audio_features,2LlQb7Uoj1kKyGhlkBf9aC,spotify:track:2LlQb7Uoj1kKyGhlkBf9aC,https://api.spotify.com/v1/tracks/2LlQb7Uoj1kK...,https://api.spotify.com/v1/audio-analysis/2LlQ...,357800,4,https://open.spotify.com/track/2LlQb7Uoj1kKyGh...
1,0.778,0.719,4,-9.698,1,0.0345,0.0123,0.026200,0.297,0.729,115.400,audio_features,569uHYIB0X324FZOBEhvit,spotify:track:569uHYIB0X324FZOBEhvit,https://api.spotify.com/v1/tracks/569uHYIB0X32...,https://api.spotify.com/v1/audio-analysis/569u...,239133,4,https://open.spotify.com/track/569uHYIB0X324FZ...
2,0.705,0.712,6,-6.156,1,0.0385,0.0102,0.000855,0.100,0.620,97.512,audio_features,2Y0iGXY6m6immVb2ktbseM,spotify:track:2Y0iGXY6m6immVb2ktbseM,https://api.spotify.com/v1/tracks/2Y0iGXY6m6im...,https://api.spotify.com/v1/audio-analysis/2Y0i...,299960,4,https://open.spotify.com/track/2Y0iGXY6m6immVb...
3,0.692,0.711,0,-7.498,0,0.0317,0.2250,0.000000,0.120,0.875,125.135,audio_features,1TfqLAPs4K3s2rJMoCokcS,spotify:track:1TfqLAPs4K3s2rJMoCokcS,https://api.spotify.com/v1/tracks/1TfqLAPs4K3s...,https://api.spotify.com/v1/audio-analysis/1Tfq...,216933,4,https://open.spotify.com/track/1TfqLAPs4K3s2rJ...
4,0.483,0.579,5,-13.923,1,0.1160,0.4810,0.001410,0.242,0.891,122.527,audio_features,2hEwjc5DZVPcC4cVVBuMr5,spotify:track:2hEwjc5DZVPcC4cVVBuMr5,https://api.spotify.com/v1/tracks/2hEwjc5DZVPc...,https://api.spotify.com/v1/audio-analysis/2hEw...,140680,4,https://open.spotify.com/track/2hEwjc5DZVPcC4c...


In [18]:
elm_features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.718580,0.710820,5.660000,-6.206850,0.640000,0.150606,0.113639,0.039119,0.199139,0.571570,121.983160,246566.850000,3.990000
std,0.149945,0.143471,3.926651,2.454781,0.482418,0.109248,0.143045,0.139647,0.149908,0.242521,30.113959,59939.827674,0.173787
min,0.218000,0.347000,0.000000,-13.923000,0.000000,0.025700,0.000187,0.000000,0.028900,0.057300,66.930000,114893.000000,3.000000
25%,0.621500,0.615250,1.750000,-7.640250,0.000000,0.066075,0.014950,0.000000,0.096425,0.429250,99.939000,207425.250000,4.000000
50%,0.732500,0.711500,5.000000,-6.048000,1.000000,0.109000,0.066800,0.000006,0.134000,0.587500,118.292500,240213.500000,4.000000
75%,0.835000,0.813500,9.000000,-4.441000,1.000000,0.231250,0.161500,0.001563,0.281750,0.759000,143.540500,276233.500000,4.000000
max,0.965000,0.993000,11.000000,-1.148000,1.000000,0.567000,0.768000,0.751000,0.890000,0.966000,210.164000,417733.000000,5.000000


In [19]:
# the total length of the playlist
total_length = elm_features['duration_ms'].sum() / 1000 / 60 / 60
print(f'The total length of the playlist is {total_length} hours')

The total length of the playlist is 6.849079166666667 hours


In [ ]:
# download first 200 songs
for track in tqdm(tracks['items']):
    s.download(track['track']['external_urls']['spotify'])

  0%|          | 0/100 [00:00<?, ?it/s]

ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
  0%|          | 0/100 [00:06<?, ?it/s]
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'


KeyboardInterrupt: 

ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion failed: Unknown encoder 'libmp3lame'
ERROR: audio conversion 

In [15]:
%matplotlib notebook

# Create a figure and axis with custom size
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.grid(True)

# Create a list to store the drawn points
drawn_points = []

# Function to handle mouse click events
def on_click(event):
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        drawn_points.append((x, y))
        ax.plot(x, y, 'ro')  # Plot the clicked point in red
        plt.draw()

# Connect the click event to the figure
fig.canvas.mpl_connect('button_press_event', on_click)

# Add a cursor to the plot
cursor = Cursor(ax, useblit=True, color='red', linewidth=1)

# Define a widget to display the drawn points
output_widget = widgets.Output()

# Function to update the widget with the drawn points
def update_output(change):
    with output_widget:
        output_widget.clear_output()
        for point in drawn_points:
            print(f'X: {point[0]}, Y: {point[1]}')

# Observe changes in the drawn points and trigger the update function
output_widget.observe(update_output)

# Display the output widget
display(output_widget)

plt.show()


<IPython.core.display.Javascript object>

Output()

In [19]:
import plotly.graph_objects as go
from ipywidgets import Output
import plotly.offline as pyo

%matplotlib notebook

# Create an output widget for displaying coordinates
out = Output()

# Create a scatter plot
scatter = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    marker=dict(size=10),
    showlegend=False,
)

fig = go.FigureWidget(data=[scatter])

# Define a callback function for handling click events
def handle_click(trace, points, state):
    if not points.xs:
        return

    x, y = points.xs[0], points.ys[0]
    with out:
        print(f'X: {x:.2f}, Y: {y:.2f}')

# Add a click event handler to the scatter plot
scatter.on_click(handle_click)

# Update the layout for better interaction
fig.update_layout(
    xaxis_title='X-axis',
    yaxis_title='Y-axis',
    autosize=True,
    margin=dict(l=0, r=0, t=30, b=30),
)

# Display the plot
pyo.iplot(fig)

# Display the output widget
out


Output()

# Intelligent Lightshow Pipeline

    - Scrape all the songs in the playlist useing the dataframe. Add a column with paths to the audio files.
    - Run BasicNote (or equivalent) on all the songs. Get corresponding midi files, add another column. 
    - Map MIDI outputs lights in EMU. 
    - Write code with a while loop that asks what song is currently playing in Spotify. When new songs start (current != previous), trigger that song to play locally. 
    - Activate corresponding MIDI file to light show. 
    -  If songs are added to the queue that are not in the dataframe already: Scrape the song, send it to BasicNote, and add it to the dataframe. 

If all this works (which it would in principal), lightshows are generated for songs added to the queue.

In [2]:
# import tekore as tk

# conf = tk.config_from_environment()
# token = tk.prompt_for_user_token(*conf)

# spotify = tk.Spotify(token)
# playlist = spotify.followed_playlists(limit=1).items[0]
# track = spotify.playlist_items(playlist.id, limit=1).items[0].track
# name = f'"{track.name}" from {playlist.name}'

# if track.episode:
#     print(f'Cannot analyse episodes!\nGot {name}.')
# elif track.track and track.is_local:
#     print(f'Cannot analyse local tracks!\nGot {name}.')
# else:
#     print(f'Analysing {name}...\n')
#     analysis = spotify.track_audio_features(track.id)
#     print(repr(analysis))

Opening browser for Spotify login...
